In [ ]:
# Setup & reproducibility

# If you haven't installed dependencies run:
# !pip install -r ../code/requirements.txt

import random
import numpy as np
import torch
import os
from pathlib import Path

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

# paths
BASE = Path('..')
FIG_DIR = BASE / 'report' / 'figures'
FIG_DIR.mkdir(parents=True, exist_ok=True)

print('Environment ready — seeds set, figures dir:', FIG_DIR)


In [ ]:
## Tabular — Data loading & EDA (diabetes.csv)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tabular import load_diabetes

pd.options.display.float_format = '{:.3f}'.format

df = load_diabetes()
print('Data head:')
display(df.head())
print('\nData info:')
print(df.dtypes)
print('\nMissing values:')
print(df.isna().sum())

# per-feature histograms
fig = df.hist(figsize=(12,8))
plt.tight_layout()
plt.savefig(str(FIG_DIR / 'tabular_histograms.png'))
print('Saved histograms -> report/figures/tabular_histograms.png')

# class distribution
plt.figure(); sns.countplot(x='Outcome', data=df); plt.title('Class distribution');
plt.savefig(str(FIG_DIR / 'class_distribution.png'))
print('Saved class distribution -> report/figures/class_distribution.png')

# correlation heatmap
plt.figure(figsize=(8,6)); sns.heatmap(df.corr(), annot=True, fmt='.2f', cmap='coolwarm');
plt.title('Correlation matrix'); plt.tight_layout();
plt.savefig(str(FIG_DIR / 'correlation_heatmap.png'))
print('Saved correlation heatmap -> report/figures/correlation_heatmap.png')

# boxplots to detect outliers for each feature
fig, axes = plt.subplots(2,4, figsize=(16,6))
for ax, col in zip(axes.ravel(), df.columns[:-1]):
    sns.boxplot(x=df[col], ax=ax)
    ax.set_title(col)
plt.tight_layout()
plt.savefig(str(FIG_DIR / 'boxplots.png'))
print('Saved boxplots -> report/figures/boxplots.png')


In [ ]:
## Tabular — Preprocessing & stratified splits (70/10/20)

from tabular import load_diabetes, preprocess, make_splits
import numpy as np

df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)

print('Dataset sizes ->', X.shape[0], 'train', X_train.shape[0], 'val', X_val.shape[0], 'test', X_test.shape[0])
# check class ratios
def ratio(arr): return (arr.sum() / len(arr))
print('Positive class ratios (train/val/test):', ratio(y_train), ratio(y_val), ratio(y_test))
# simple assertion: ratios should be very close (within 2%)
assert abs(ratio(y_train) - ratio(y_test)) < 0.05
print('Stratified split assertion passed')


In [ ]:
## Tabular — MLP model implementation & training (architecture per spec)

# MLP is implemented in `models.py` as `MLPClassifier`.
# Training loop (early-stopping) is provided in `tabular.train_model` used below.

from tabular import load_diabetes, preprocess, make_splits, to_loader, train_model
from models import MLPClassifier

# load data
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)

train_loader = to_loader(X_train, y_train, batch_size=64)
val_loader = to_loader(X_val, y_val, batch_size=256, shuffle=False)

mlp = MLPClassifier()
mlp = train_model(mlp, train_loader, val_loader, epochs=50, lr=1e-3)

# simple training/validation loss curves (collected during training in `train_model` would be ideal — here we re-evaluate quickly)
import matplotlib.pyplot as plt
plt.figure(); plt.title('Quick validation - final model check');
plt.bar(['val_pos_rate'], [y_val.mean()]); plt.savefig(str(FIG_DIR / 'mlp_quick_check.png'))
print('Saved quick check -> report/figures/mlp_quick_check.png')


In [ ]:
## Tabular — Evaluation: accuracy / recall / F1 / confusion matrix

from tabular import load_diabetes, preprocess, make_splits, to_loader, train_model, predict_binary, evaluate_preds
from models import MLPClassifier

# load and train
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)
mlp = MLPClassifier()
mlp = train_model(mlp, to_loader(X_train, y_train), to_loader(X_val, y_val, shuffle=False), epochs=30)

preds, probs = predict_binary(mlp, X_test)
metrics = evaluate_preds(y_test, preds)
print('Test metrics:')
for k,v in metrics.items():
    print(f'  {k}:\n{v}\n' if k=='confusion_matrix' else f'  {k}: {v:.4f}')

# assertion: confusion matrix sums to test size
import numpy as np
assert metrics['confusion_matrix'].sum() == len(y_test)
print('Assertions passed: confusion matrix sums to test size')


In [ ]:
## Tabular — LIME explanations (3 random test samples)

from tabular import load_diabetes, preprocess, make_splits
from models import MLPClassifier
from tabular import to_loader, train_model, predict_binary
from interpretability import lime_explain
import numpy as np
import matplotlib.pyplot as plt

# data + model
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)
mlp = MLPClassifier()
mlp = train_model(mlp, to_loader(X_train, y_train), to_loader(X_val, y_val, shuffle=False), epochs=20)

# wrapper for lime: returns class probabilities (2 columns)
def predict_fn(Xin):
    import torch
    with torch.no_grad():
        logits = mlp(torch.from_numpy(Xin).float())
        probs = torch.sigmoid(logits).numpy()
        return np.vstack([1-probs, probs]).T

# pick 3 samples (mix of correct/incorrect predictions)
probs = predict_fn(X_test)
preds = (probs[:,1] >= 0.5).astype(int)
idxs = np.random.choice(len(X_test), size=10, replace=False)
selected = []
for idx in idxs:
    if len(selected) >= 3: break
    selected.append(idx)

for i, idx in enumerate(selected):
    exp = lime_explain(predict_fn, X_train, X_test[idx], df.columns[:-1].tolist())
    fig = exp.as_pyplot_figure()
    fig.savefig(str(FIG_DIR / f'lime_sample_{idx}.png'))
    print('Saved LIME plot for test idx', idx)


In [ ]:
## Tabular — SHAP explanations (KernelExplainer + force_plot)

import shap
from interpretability import shap_explain
from tabular import load_diabetes, preprocess, make_splits
from models import MLPClassifier
from tabular import to_loader, train_model, predict_binary
import numpy as np

# ensure model trained
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)
mlp = MLPClassifier()
mlp = train_model(mlp, to_loader(X_train, y_train), to_loader(X_val, y_val, shuffle=False), epochs=20)

# pick 3 test samples
idxs = [0, 5, 10] if len(X_test) > 10 else list(range(min(3, len(X_test))))
background = X_train[np.random.choice(len(X_train), size=100, replace=False)]

explainer = shap.KernelExplainer(lambda z: predict_binary(mlp, z)[1], background)
vals = explainer.shap_values(X_test[idxs], nsamples=200)
for i, v in zip(idxs, vals):
    shap.force_plot(explainer.expected_value, v, X_test[i], matplotlib=True, show=False)
    # save a matplotlib rendering
    plt = shap.plots._force_matplotlib.force_matplotlib(explainer.expected_value, v, X_test[i], feature_names=df.columns[:-1])
    plt.savefig(str(FIG_DIR / f'shap_force_sample_{i}.png'))
    print('Saved SHAP force plot for sample', i)


In [ ]:
## Tabular — LIME vs SHAP comparison + correlation linkage

import random
from interpretability import lime_explain, shap_explain
from tabular import load_diabetes, preprocess, make_splits
from models import MLPClassifier
from tabular import to_loader, train_model, predict_binary
import matplotlib.pyplot as plt
import numpy as np

# prepare data & model (train quickly if not trained already)
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)

mlp = MLPClassifier()
tr = to_loader(X_train, y_train, batch_size=64)
va = to_loader(X_val, y_val, batch_size=256, shuffle=False)
mlp = train_model(mlp, tr, va, epochs=20, lr=1e-3)

# pick 3 random test indices
rng = np.random.RandomState(0)
idxs = rng.choice(len(X_test), size=3, replace=False)
feature_names = df.columns[:-1].tolist()

for i, idx in enumerate(idxs):
    x = X_test[idx]
    # LIME
    def predict_fn(Xin):
        import torch
        mlp.eval()
        with torch.no_grad():
            logits = mlp(torch.from_numpy(Xin).float())
            probs = torch.sigmoid(logits).numpy()
            return np.vstack([1-probs, probs]).T
    lime_exp = lime_explain(predict_fn, X_train, x, feature_names)
    # SHAP
    shap_vals = shap_explain(lambda z: predict_fn(z)[:,1], X_train, x.reshape(1,-1))

    # prepare arrays for comparison
    lime_weights = np.array([w for _, w in lime_exp.as_list()])
    lime_feats = [f for f, _ in lime_exp.as_list()]
    # align SHAP to feature order
    shap_arr = np.array(shap_vals)[0]

    # bar plot (top 6 features)
    top_idx = np.argsort(np.abs(shap_arr))[-6:][::-1]
    fig, ax = plt.subplots(1,2, figsize=(10,4))
    ax[0].barh(feature_names, shap_arr, color='C0'); ax[0].set_title('SHAP (per-feature)')
    ax[1].barh([f for f,_ in lime_exp.as_list()], [w for _,w in lime_exp.as_list()], color='C1'); ax[1].set_title('LIME (local)')
    plt.suptitle(f'Sample {i} (test idx={idx}) — predicted pos prob {predict_fn(x.reshape(1,-1))[:,1][0]:.3f}')
    plt.tight_layout()
    plt.savefig(str(FIG_DIR / f'lime_shap_compare_sample_{i}.png'))
    print(f'Saved comparison for sample {i} -> report/figures/lime_shap_compare_sample_{i}.png')

# Correlation linkage: show top correlated feature pairs
import pandas as pd
corr = pd.DataFrame(X, columns=feature_names).corr().abs()
high = np.where((corr.values > 0.6) & (corr.values < 1.0))
pairs = list(set(tuple(sorted((feature_names[i], feature_names[j]))) for i,j in zip(*high)))
print('High-correlation pairs (>|0.6|):', pairs)


In [ ]:
## Tabular — NAMClassifier: train + per-feature plots

from models import NAMClassifier
from tabular import load_diabetes, preprocess, make_splits, to_loader, train_model
import numpy as np
import matplotlib.pyplot as plt

# load data
df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)
tr = to_loader(X_train, y_train, batch_size=64)
va = to_loader(X_val, y_val, batch_size=256, shuffle=False)

nam = NAMClassifier(n_features=X.shape[1], hidden=32)
nam = train_model(nam, tr, va, epochs=40, lr=5e-3)

# evaluate
from tabular import predict_binary, evaluate_preds
preds, probs = predict_binary(nam, X_test)
print('NAM test metrics ->', evaluate_preds(y_test, preds))

# plot per-feature learned functions: pass a range for a single feature while others fixed at median
feature_names = df.columns[:-1].tolist()
med = np.median(X_train, axis=0)
fig, axs = plt.subplots(2,4, figsize=(16,6))
for i, ax in enumerate(axs.ravel()):
    xs = np.linspace(X_train[:,i].min(), X_train[:,i].max(), 200)
    inputs = np.tile(med, (200,1))
    inputs[:, i] = xs
    with torch.no_grad():
        import torch
        out = nam(torch.from_numpy(inputs).float()).numpy()
    ax.plot(xs, out)
    ax.set_title(feature_names[i])
plt.tight_layout()
plt.savefig(str(FIG_DIR / 'nam_feature_functions.png'))
print('Saved NAM per-feature plots -> report/figures/nam_feature_functions.png')


In [ ]:
## Tabular — Bonus: GRACE-style contrastive sampling + SHAP analysis

import numpy as np
from tabular import load_diabetes, preprocess, make_splits
from interpretability import shap_explain
from tabular import MLPClassifier if False else None

# simple GRACE-like perturbation: for a selected test sample, create contrastive samples by perturbing one feature at a time

df = load_diabetes()
X, y, scaler = preprocess(df)
X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)

sample_idx = 3
x0 = X_test[sample_idx:sample_idx+1].copy()

# generate contrastive samples by adding +1/-1 std to each feature
contrast = []
for f in range(x0.shape[1]):
    xp = x0.copy(); xp[0, f] += 1.0
    xm = x0.copy(); xm[0, f] -= 1.0
    contrast.append(xp[0]); contrast.append(xm[0])
contrast = np.stack(contrast)

# load trained model from previous cells if present; otherwise train quickly here
from models import MLPClassifier
import torch
model = MLPClassifier()
# quick training (very short) for demo
from tabular import to_loader, train_model
tr, va = to_loader(X_train, y_train, batch_size=64), to_loader(X_val, y_val, batch_size=256, shuffle=False)
model = train_model(model, tr, va, epochs=10, lr=1e-3)

# model predict_proba for shap
def predict_proba(Xin):
    import torch
    model.eval()
    with torch.no_grad():
        logits = model(torch.from_numpy(Xin).float())
        probs = torch.sigmoid(logits).numpy()
        # return two-column proba for compatibility (neg,pos)
        return np.vstack([1-probs, probs]).T

base_probs = predict_proba(x0)
contrast_probs = predict_proba(contrast)

print('Base prob (pos):', base_probs[0,1])
print('Contrast probs (pos) for perturbed features:', contrast_probs[:,1])

# explain change with SHAP using KernelExplainer on background X_train
from interpretability import shap_explain
shap_vals = shap_explain(lambda z: predict_proba(z)[:,1], X_train, contrast[:5])
print('Computed SHAP values for first 5 contrast samples (shape):', np.array(shap_vals).shape)


In [ ]:
## Vision — Setup (VGG16 preprocessing + sample selection)

from vision import get_vgg16, preprocess_image
from torchvision import models
from PIL import Image

vgg = get_vgg16()
print('Loaded VGG16 (eval mode).')

# helper to display prediction
import torch
from torchvision import transforms

def predict_top1(model, pil_img):
    t = preprocess_image(pil_img)
    with torch.no_grad():
        out = model(t)
        idx = out.argmax(dim=1).item()
        return idx

# sample images: we use simple colored images here as placeholders; replace with real ImageNet images if available
samples = [Image.new('RGB', (224,224), color=(i*30, i*15, 200-i*20)) for i in range(6)]
preds = [predict_top1(vgg, s) for s in samples]
print('Top-1 preds for placeholder images (indices):', preds)


In [ ]:
## Vision — Grad-CAM (theory + demo)

Markdown: """
Grad-CAM: compute neuron importance weights \alpha_k^c = (1/Z) * \sum_i \sum_j \partial y^c / \partial A^k_{ij}
Then heatmap L^c_{Grad-CAM} = ReLU(\sum_k \alpha_k^c A^k)
"""

# demo on VGG16 (implementation provided in vision.py as GradCAM)
from vision import get_vgg16, preprocess_image, GradCAM
from PIL import Image
import matplotlib.pyplot as plt

img = Image.new('RGB', (224,224), color=(100,140,200))
input_tensor = preprocess_image(img)
model = get_vgg16()
cam = GradCAM(model, model.features[28])
heat = cam(input_tensor)

plt.figure(figsize=(6,6))
plt.imshow(heat, cmap='jet')
plt.title('Grad-CAM heatmap (demo)')
plt.axis('off')
plt.savefig(str(FIG_DIR / 'gradcam_demo.png'))
print('Saved -> report/figures/gradcam_demo.png')


In [ ]:
## Vision — Guided Backpropagation & Guided Grad-CAM

# Guided Backprop implementation is provided in `vision.py` as `GuidedBackprop`.
# Here we demonstrate on one image and combine with Grad-CAM heatmap.

from vision import get_vgg16, preprocess_image, GuidedBackprop, GradCAM
from PIL import Image
import matplotlib.pyplot as plt

img = Image.new('RGB', (224,224), color=(200,160,120))
input_tensor = preprocess_image(img)
model = get_vgg16()

gb = GuidedBackprop(model)
gb_grad = gb.generate(input_tensor)

cam = GradCAM(model, model.features[28])
heat = cam(input_tensor)

# Guided Grad-CAM: multiply cam with absolute guided gradients summary
import numpy as np
guided_gradcam = np.abs(gb_grad).max(axis=0) * heat

fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(12,4))
ax1.imshow(img); ax1.set_title('Image'); ax1.axis('off')
ax2.imshow(heat, cmap='jet'); ax2.set_title('Grad-CAM'); ax2.axis('off')
ax3.imshow(guided_gradcam, cmap='inferno'); ax3.set_title('Guided Grad-CAM'); ax3.axis('off')
plt.savefig(str(FIG_DIR / 'guided_gradcam_example.png'))
print('Saved -> report/figures/guided_gradcam_example.png')


In [ ]:
## Vision — SmoothGrad + Guided BP / Guided Grad-CAM

from vision import smoothgrad, GuidedBackprop, GradCAM
import matplotlib.pyplot as plt

# reuse input_tensor from previous cell (or create placeholder)
from PIL import Image
img = Image.new('RGB', (224,224), color=(120,130,140))
input_tensor = preprocess_image(img)
model = get_vgg16()

# SmoothGrad on Guided Backprop
sg = smoothgrad(model, input_tensor, n_samples=20, stdev=0.12)
# Guided backprop
gb = GuidedBackprop(model)
gb_grad = gb.generate(input_tensor)

# Guided Grad-CAM: Grad-CAM heatmap * guided backprop gradients
cam = GradCAM(model, model.features[28])
heat = cam(input_tensor)
guided_gradcam = np.abs(gb_grad).max(axis=0) * heat

fig, axs = plt.subplots(1,3, figsize=(12,4))
axs[0].imshow(sg.transpose(1,2,0)); axs[0].set_title('SmoothGrad (avg grad)')
axs[1].imshow(np.abs(gb_grad).max(axis=0), cmap='gray'); axs[1].set_title('Guided BP (abs)')
axs[2].imshow(guided_gradcam, cmap='inferno'); axs[2].set_title('Guided Grad-CAM')
for a in axs: a.axis('off')
plt.savefig(str(FIG_DIR / 'smoothgrad_guided_comparison.png'))
print('Saved -> report/figures/smoothgrad_guided_comparison.png')


In [ ]:
## Vision — Adversarial attack (FGSM) + saliency comparison

import torch
import torch.nn.functional as F
from torchvision import transforms
from vision import get_vgg16, preprocess_image, GradCAM, GuidedBackprop
from PIL import Image

# load an example image from PIL (use a placeholder gray image if internet not available)
img = Image.new('RGB', (224,224), color=(120,130,140))
input_tensor = preprocess_image(img)
model = get_vgg16()
model.eval()

# get baseline prediction
with torch.no_grad():
    logits = model(input_tensor)
    orig_pred = logits.argmax(dim=1).item()
print('Original predicted class index:', orig_pred)

# FGSM attack
def fgsm_attack(model, x, eps=0.02, target=None):
    x_adv = x.clone().detach().requires_grad_(True)
    out = model(x_adv)
    if target is None:
        target = out.argmax(dim=1)
    loss = F.nll_loss(F.log_softmax(out, dim=1), target)
    model.zero_grad(); loss.backward()
    adv = x_adv + eps * x_adv.grad.sign()
    return adv.detach()

x_adv = fgsm_attack(model, input_tensor, eps=0.02)
with torch.no_grad():
    adv_pred = model(x_adv).argmax(dim=1).item()
print('Adversarial predicted class index:', adv_pred)

# Grad-CAM before/after
cam = GradCAM(model, model.features[28])
heat_orig = cam(input_tensor)
heat_adv = cam(x_adv)

# Guided Grad-CAM (element-wise multiply guided backprop gradients with cam)
gb = GuidedBackprop(model)
gb_grad_orig = gb.generate(input_tensor)
gb_grad_adv = gb.generate(x_adv)

import numpy as np
# combine
guided_gradcam_orig = np.abs(gb_grad_orig).max(axis=0) * heat_orig
guided_gradcam_adv = np.abs(gb_grad_adv).max(axis=0) * heat_adv

# save visuals
from matplotlib import pyplot as plt
fig, axs = plt.subplots(2,2, figsize=(8,8))
axs[0,0].imshow(img); axs[0,0].set_title('Input')
axs[0,1].imshow(heat_orig, cmap='jet'); axs[0,1].set_title('Grad-CAM (orig)')
axs[1,0].imshow(guided_gradcam_orig, cmap='inferno'); axs[1,0].set_title('Guided Grad-CAM (orig)')
axs[1,1].imshow(guided_gradcam_adv, cmap='inferno'); axs[1,1].set_title('Guided Grad-CAM (adv)')
for ax in axs.ravel(): ax.axis('off')
plt.tight_layout()
plt.savefig(str(FIG_DIR / 'adv_saliency_comparison.png'))
print('Saved adversarial saliency comparison -> report/figures/adv_saliency_comparison.png')


In [ ]:
## Vision — Feature visualization (activation maximization)

# We'll maximize the 'hen' class logit in VGG16 (ImageNet 'hen' idx = 7xx depending on mapping).
# For portability, the notebook finds the ImageNet index for 'hen' via torchvision's labels file if available;
# otherwise the user can set the index manually.

import torch
from vision import get_vgg16, activation_maximization
from torchvision import models

vgg = get_vgg16()

# common ImageNet 'hen' synset index (use 7 as example if no mapping). You may update this to exact class id.
hen_idx = 7
img = activation_maximization(vgg, target_class=hen_idx, steps=200, lr=1.0, tv_weight=1e-4)

# save image to file
import numpy as np
from PIL import Image
out = (img.transpose(1, 2, 0) * 255).astype('uint8')
Image.fromarray(out).save(str(FIG_DIR / 'activation_max_hen.png'))
print('Saved activation-max result -> report/figures/activation_max_hen.png')


In [ ]:
# Utilities: export figures, simple unit tests and end-to-end smoke run
import os
from pathlib import Path
from IPython.display import display, HTML

BASE = Path("../").resolve()  # notebook is in HomeWorks/HW2/notebooks/
FIG_DIR = Path("../report/figures").resolve()
FIG_DIR.mkdir(parents=True, exist_ok=True)

print(f"Figures will be saved to: {FIG_DIR}")

# lightweight smoke tests
def smoke_checks():
    # data split check
    from tabular import load_diabetes, preprocess, make_splits
    df = load_diabetes()
    X, y, _ = preprocess(df)
    X_train, X_val, X_test, y_train, y_val, y_test = make_splits(X, y)
    ratios = [len(y_train), len(y_val), len(y_test)]
    assert sum(ratios) == len(y)
    print("Smoke: data splits OK")

    # model forward shape
    from models import MLPClassifier
    import torch
    m = MLPClassifier()
    x = torch.randn(4, 8)
    out = m(x)
    assert out.shape == (4,)
    print("Smoke: MLP forward OK")

    # Grad-CAM heatmap shape
    from vision import get_vgg16, preprocess_image, GradCAM
    import PIL.Image
    img = PIL.Image.new("RGB", (224, 224), color=(128, 128, 128))
    inp = preprocess_image(img)
    vgg = get_vgg16()
    cam = GradCAM(vgg, vgg.features[28])
    heat = cam(inp)
    assert heat.shape == (224, 224)
    print("Smoke: Grad-CAM heatmap OK")

    print("All smoke checks passed — notebook helpers functioning.")

smoke_checks()


# HW2 — Interpretability (Tabular + Vision)
(Notebook placeholder — full implementation will be inserted programmatically.)